在 CPU 版本和 GPU 版本的代码相比，主要区别如下：

1. **模型转移到设备**

   - **CPU 版本的第4行**：无此步骤，模型默认在 CPU 上。
   - **GPU 版本的第4行**：`device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')` 增加了 GPU 设备选择的步骤。
   - **GPU 版本的第5行**：`model.to(device)` 增加了将模型转移到 GPU 的步骤。

2. **数据转移到设备**

   - **CPU 版本的第14行**：无此步骤，数据默认在 CPU 上。
   - **GPU 版本的第14行**：`images, labels = images.to(device), labels.to(device)` 增加了将数据转移到 GPU 的步骤。

3. **训练过程中数据转移**

   - **CPU 版本的第28行**：无此步骤，数据在 CPU 上。
   - **GPU 版本的第28行**：`images, labels = images.to(device), labels.to(device)` 增加了将数据转移到 GPU 的步骤。

4. **评估过程中数据转移**

   - **CPU 版本的第37行**：无此步骤，数据在 CPU 上。
   - **GPU 版本的第37行**：`images, labels = images.to(device), labels.to(device)` 增加了将数据转移到 GPU 的步骤。

5. **预测单个样本**

   - **CPU 版本的第46行**：无此步骤，数据在 CPU 上。
   - **GPU 版本的第46行**：`image = images[0].unsqueeze(0).to(device)` 增加了将数据转移到 GPU 的步骤。

1. 设计一个具有单隐层的多层感知器网络；
2. 构造多层感知器前向传播和后向传播程序；
3. 训练多层感知器；
4. 利用多层感知器实现 MINIST 手写体识别；

# 1 数据准备

In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

# 设置超参数
batch_size = 64
learning_rate = 0.01
num_epochs = 10

# 定义数据预处理方式
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 正则化数据
])

# 加载训练集和测试集
train_dataset = datasets.MNIST(root='../data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='../data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 2 定义模型

In [ ]:
import torch
from torch import nn

class MyMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.cf1 = nn.Linear(28*28, 128)
        self.cf2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.cf1(x))
        x = self.cf2(x)
        return x

# 初始化模型
model = MyMLP()

# 2.5 检查 GPU 是否可用并将模型转移到 GPU 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

model.to(device)  # 将模型转移到 GPU

# 3 定义损失函数和优化器

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 4 训练模型

In [ ]:
model.train()
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        images, labels = images.to(device), labels.to(device)  # 将数据转移到 GPU

        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 后向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print(f'Epoch: {epoch+1}; Step: {i+1}; Loss: {loss.item():.4f}.')  # 打印损失

# 5 评估模型

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():  # 不需要计算梯度
    for images, labels in test_loader:

        images, labels = images.to(device), labels.to(device)  # 将数据转移到 GPU

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = correct / total

print(f'Accuracy: {accuracy*100:.2f}%.')  # 打印准确率

# 6 预测单个样本（附加）

In [ ]:
import matplotlib.pyplot as plt

# 从测试集中取一个样本
sample = next(iter(test_loader))
images, labels = sample

image = images[0].unsqueeze(0).to(device)  # 增加一个维度，适应模型输入，并转移到 GPU

# 取第一张图片进行预测
model.eval()
with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output.data, 1)

# 可视化样本及其预测结果
plt.imshow(images[0].squeeze(), cmap='gray')
plt.title(f'Predicted: {predicted.item()}')
plt.show()